In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil

root_dir = "/content/drive/MyDrive/ReID_data/DukeMTMC-reID/bounding_box_train"
output_dir = "/content/train_split"

os.makedirs(output_dir, exist_ok=True)

for img_name in os.listdir(root_dir):
    if not img_name.endswith(".jpg"):
        continue
    person_id = img_name[:4]  # first 4 chars are ID
    person_dir = os.path.join(output_dir, person_id)
    os.makedirs(person_dir, exist_ok=True)
    shutil.copy(os.path.join(root_dir, img_name), os.path.join(person_dir, img_name))

print("✅ Images organized into folders by person ID.")

✅ Images organized into folders by person ID.


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paths
train_path = "/content/train_split"

# Transform
transform = transforms.Compose([
    transforms.Resize((128, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3),
])

# Dataset & Dataloader
dataset = ImageFolder(train_path, transform=transform)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=2)

# Model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(dataset.classes))  # num classes = num IDs
model = model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 70.4MB/s]


Epoch 1/10, Loss: 223.0834
Epoch 2/10, Loss: 95.9078
Epoch 3/10, Loss: 45.1504
Epoch 4/10, Loss: 19.5225
Epoch 5/10, Loss: 8.7925
Epoch 6/10, Loss: 4.6591
Epoch 7/10, Loss: 3.1931
Epoch 8/10, Loss: 2.2285
Epoch 9/10, Loss: 1.6652
Epoch 10/10, Loss: 1.3611


In [4]:
save_path = "/content/drive/MyDrive/reid_model.pt"
torch.save(model.state_dict(), save_path)
print(f"✅ Model saved to: {save_path}")

✅ Model saved to: /content/drive/MyDrive/reid_model.pt
